In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

def load_image(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device)


In [ ]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        #Running only till 28th module in vgg. Ignoring fully connected layers. 
        self.chosen_features = ["0", "5", "10", "19", "28"]
        self.model = models.vgg19(pretrained=True).features[:29]

    def forward(self, x):
        features = []

        # Go through each layer in model. 
        for layer_num, layer in enumerate(self.model):
            x = layer(x)
            #Check if layer is in the chosen features. 
            #Return all the activations for the specific layers we have in chosen_features
            if str(layer_num) in self.chosen_features:
                features.append(x)

        return features



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
imsize = 356

loader = transforms.Compose(
    [
        transforms.Resize((imsize, imsize)),
        transforms.ToTensor(),
    ]
)

original_img = load_image("image.jpg")
style_img = load_image("style.jpg")

# initialized generated as white noise or clone of original image.
# generated = torch.randn(original_img.data.shape, device=device, requires_grad=True)
generated = original_img.clone().requires_grad_(True)
model = VGG().to(device).eval()

# Hyperparameters
total_steps = 6000
learning_rate = 0.001
alpha = 1
beta = 0.01
optimizer = optim.Adam([generated], lr=learning_rate)

for step in range(total_steps):
    # Obtain the convolution features in specifically chosen layers
    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)

    # Loss is 0 initially
    style_loss = original_loss = 0

    # iterate through all the features for the chosen layers
    for gen_feature, orig_feature, style_feature in zip(
        generated_features, original_img_features, style_features
    ):

        batch_size, channel, height, width = gen_feature.shape
        #Content loss
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)

        # Compute Gram Matrix of generated
        G = gen_feature.view(channel, height * width).mm(
            gen_feature.view(channel, height * width).t()
        )
        # Compute Gram Matrix of Style
        A = style_feature.view(channel, height * width).mm(
            style_feature.view(channel, height * width).t()
        )
        #Style loss
        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(total_loss)
        save_image(generated, "generated.png")

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

tensor(1457714.2500, grad_fn=<AddBackward0>)
tensor(71186.4219, grad_fn=<AddBackward0>)
tensor(34013.5859, grad_fn=<AddBackward0>)
tensor(24063.2949, grad_fn=<AddBackward0>)
tensor(19353.5039, grad_fn=<AddBackward0>)
tensor(16428.1035, grad_fn=<AddBackward0>)
tensor(14320.7256, grad_fn=<AddBackward0>)
tensor(12659.6045, grad_fn=<AddBackward0>)
